# rospy Snippets

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/AIResearchLab/foundations-of-robotics-labs/blob/master/1-sense-think-act/02-rospy-snippets.ipynb)
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/AIResearchLab/foundations-of-robotics-labs/master?filepath=1-sense-think-act/02-rospy-snippets.ipynb)

The Sphero RVR is a great robot for learning some robotics, but how do we work with ROS? This tutorial will show you how to use snippets of ROS code using rospy in python to interact with the RVR.

### Testing the Snippets

To test the snippets, you will need to have the RVR running. You will also need to have the `sphero_rvr_ros` package installed and running the main application stack.

```bash
# in a terminal on the raspberry pi onboard the RVR
# as a reminder, you can run the main application stack with
roslaunch sphero_rvr_deployment sphero_rvr_deployment.launch
```

> **Hints**
> - You can then run the snippets in a jupyter notebook or in a python file.
> - You can open terminals in jupyter environment by clicking `File > New > Terminal` in the top left corner of the jupyter notebook.
> - to run a python file, you can use the command `python <filename>.py` in the terminal.


## Receiving Messages

A core part of ROS programming is the use of ***`callback`*** functions. In ROS, A callback function is executed when a message is received on a topic. This is common place in many programs that perform communications. Messaging applications such as Messenger, Slack, or Discord also use a callback pattern pass to messages around the world.

The following code snippet shows how to create a callback function, subscribe to a topic, and execute the callback function when a message is received.

In [ ]:
"""receiving message data"""


# import python libraries
import rospy # ros library for python
from sensor_msgs.msg import Imu # import the Imu message type definition


# create a class to subscribe to sensor messages and show the data
class SubscribeAndShowNode(object):
    def __init__(self):
        # subscribe to the topic ~imu from the rvr namespace
        self.imu_sub = rospy.Subscriber('/rvr_driver/imu', Imu, self.callback)

    # callback function to receive the data
    def callback(self, msg):
        # print the data received
        rospy.loginfo(rospy.get_caller_id() + 'IMU DATA FROM FRAME: %s', msg.header.frame_id)
        rospy.loginfo(rospy.get_caller_id() + 'LINEAR ACCELERATION: %s', msg.linear_acceleration)


# main function
def main():
    try:
        # initialize the node with rospy
        rospy.init_node('subscribe_and_show_node')

        # create an instance of the SubscribeAndShowNode class
        # this will subscribe to the topic and show the data
        sub_show_node = SubscribeAndShowNode()

        # spin() simply keeps python from exiting until this node is stopped
        rospy.spin()

    except rospy.ROSInterruptException as e:
        print(e)
    finally:
        print('Node has shutdown')

print('the SubscribeAndShowNode class has been declared')

#### Test out subscribing to a topic

Run the following code snippet to see how to subscribe to a topic and print out the message. Inspect the ROS Network Graph to see the node that was created and the topic it is subscribing to.

> **Tip**
> - The following cell will run forever. You can stop it by clicking the stop button in the toolbar at the top of the notebook.

In [ ]:
"""run the main function to start the node and test it"""
main()

## Sending Commands

Sending commands in a ROS application is very similar to receiving messages but instead of reveiving messages, you are sending them. Another component in the system will listen for and receive these messages. The following code snippet shows how to create a ***`publisher`***, publish a message, and shutdown the publisher.

In [ ]:
"""sending commands to the robot to move it"""


# import python libraries
import rospy # ros library for python
from geometry_msgs.msg import Twist # import the Twist message type definition


# create a class to send commands to the robot
class PublishTwistCommand(object):
    # the constructor
    # the default value for the angular speed is 0.0
    # change this value when instantiating the class to make the robot turn
    def __init__(self, angular_speed=0.0):
        # create a publisher to send commands to the robot
        self.cmd_vel_pub = rospy.Publisher('/cmd_vel', Twist, queue_size=10)

        # create a Twist message
        self.twist = Twist()

        # create a rate to publish the data
        self.rate = rospy.Rate(10)

        # create a variable to hold the angular speed
        self.angular_speed = angular_speed

    def spin(self):
        # print a message
        print('starting the PublishTwistCommand node')

        # loop until the node is shutdown
        while not rospy.is_shutdown():
            # set the linear and angular speeds in the Twist message
            self.twist.linear.x = 0.0
            self.twist.angular.z = self.angular_speed

            # publish the Twist message
            self.cmd_vel_pub.publish(self.twist)

            # sleep for the remainder of the loop
            self.rate.sleep()


# main function
def main():
    try:
        # initialize the node with rospy
        rospy.init_node('publish_twist_command_node')

        # create an instance of the PublishTwistCommand class
        # this will publish the Twist message
        ptc = PublishTwistCommand(0.2) # change the angular speed to make the robot turn

        # this is an example of a manual loop in ROS
        # this also keeps python from exiting until this node is stopped
        # but it blocks execution until the node is stopped
        ptc.spin()

    except rospy.ROSInterruptException as e:
        print(e)
    finally:
        print('Node has shutdown')

print('the PublishTwistCommand class has been declared')

#### Test out publishing to a topic

Run the following code snippet to see how to publish to a topic. Inspect the ROS Network Graph to see the node that was created and the topic it is publishing to.

In [ ]:
# run the main function to start the node and test it
main()

## ROS-Style Loops

In robotics it is common to have a loop of activities that runs continuously at regular intervals. This is often referred to as a loop in programming. The best practice for creating loops in a ROS application is to use a ***`Timer`***. The following code snippet shows how to create a ROS-Style loop using a ROS Timer.

In [ ]:
"""timer based loop that cycles LED colours"""


import rospy # ros library for python
from std_msgs.msg import ColorRGBA # import the ColorRGBA message type definition
import random # python library to generate random numbers


# create a class to send commands to the robot
class TimerBasedLoopNode():
    def __init__(self):
        # create a timer to call the callback function every 1 second
        self.timer = rospy.Timer(rospy.Duration(1.0), self.callback)

    # callback function to send commands to the robot
    def callback(self, event):
        # create a publisher to send commands to the robot
        pub = rospy.Publisher('/rvr_driver/headlight', ColorRGBA, queue_size=10)

        # create a message
        msg = ColorRGBA()

        # add a random colour to the message
        msg.r = random.random()
        msg.g = random.random()
        msg.b = random.random()

        # publish the message
        pub.publish(msg)


# main function
def main():
    try:
        # initialize the node with rospy
        rospy.init_node('timer_based_loop_node')

        # create an instance of the TimerBasedLoopNode class
        # this will publish the ColorRGBA message
        tbn = TimerBasedLoopNode()

        # spin() simply keeps python from exiting until this node is stopped
        rospy.spin()

    except rospy.ROSInterruptException as e:
        print(e)
    finally:
        print('Node has shutdown')

print('the TimerBasedLoopNode class has been declared')

#### Test out a ROS-Style Loop

Run the following code snippet to see how to create a ROS-Style loop. Inspect the ROS Network Graph to see the node that was created and the topic it is publishing to.

In [ ]:
# run the main function to start the node and test it
main()

## Putting it all together

Now that we have seen how to receive messages, send commands, and create ROS-style loops, we can put it all together to create a simple ROS program. The following code snippet shows how to create a ROS node that receives messages from the RVR's colour sensor and sends commands to the RVR's LEDs.

In [ ]:
"""ROS node that cycles LED colours based on a colour sensor"""


import rospy # ros library for python
from std_msgs.msg import ColorRGBA # import the ColorRGBA message type definition


# create a class to represent the ROS Node
class ObservedColourProjectorNode():
    def __init__(self):
        # subscribe to the topic ~ground_color from the rvr namespace
        self.ground_color_sub = rospy.Subscriber('/rvr_driver/ground_color', ColorRGBA, self.ground_color_cb)

        # create a publisher to send commands to the robot
        self.headlight_color_pub = rospy.Publisher('/rvr_driver/led/headlight', ColorRGBA, queue_size=10)

        # create a timer to manage the state of the node
        # (hint) Sometimes node logic is implemented in subscriber callbacks but this is not recommended if the logic is complex or time consuming.
        # Timers are a better place to implement node logic.
        # this is a typical place to see node logic such as state machines
        self.timer = rospy.Timer(rospy.Duration(1.0), self.node_state_timer_cb)

        # state variable to keep track of the current colour
        self.current_color = ColorRGBA()

    # callback function to receive the data
    def ground_color_cb(self, msg):
        # print the data received
        rospy.loginfo(rospy.get_caller_id() + 'COLOUR SENSOR DATA: %s', msg)

        # store the data in the state variable
        self.current_color = msg

    # timer callback function to manage the state of the node
    def node_state_timer_cb(self, event):
        # create a message
        msg = ColorRGBA()

        # add the current sensed colour to the message
        msg.r = self.current_color.r
        msg.g = self.current_color.g
        msg.b = self.current_color.b

        # publish the message
        self.headlight_color_pub.publish(msg)


# main function
def main():
    try:
        # initialize the node with rospy
        rospy.init_node('observed_colour_projector_node')

        # create an instance of the ObservedColourProjectorNode class
        # this will subscribe to the topic and show the data
        ocpn = ObservedColourProjectorNode()

        # spin() simply keeps python from exiting until this node is stopped
        rospy.spin()

    except rospy.ROSInterruptException as e:
        print(e)
    finally:
        print('Node has shutdown')

print('the ObservedColourProjectorNode class has been declared')

#### Test out the ROS Program

Run the following code snippet to see how to create a ROS program. Inspect the ROS Network Graph to see the node that was created and the topics it is subscribing and publishing to.

In [ ]:
# run the main function to start the node and test it
main()